# SMP Parallelism. A highly scalable (i.e., parallelizable) isosurface contour filter for large data.


## Tested ParaView 5.10 Wed Mar 23 15:55:39 CET 2022

In [ ]:
from paraview.simple import *
from paraview.modules.vtkRemotingCore import vtkProcessModule
# Tested eiger Thu Feb 17 09:20:06 CET 2022
# Tested daint Thu Mar 10 08:51:11 CET 2022

In [ ]:
view = GetRenderView()
view.ViewSize=[512,512]

In [ ]:
reader = ImageReader(FileNames=['/scratch/snx3000/jfavre/50.density-log_1024x1024x2048_float.raw'])
reader.DataScalarType = 'float'
reader.DataByteOrder = 'LittleEndian'
reader.DataExtent = [0, 2047, 0, 1023, 0, 1023]
# we force the reading of the data to factor out the I/O time
reader.UpdatePipeline()

Outline = Show(reader, view)
Outline.Representation = 'Outline'
Outline.ColorArrayName = ['POINTS', '']

In [ ]:
%%time
contour1 = Contour(registrationName='Contour1', Input=reader)
contour1.ContourBy = ['POINTS', 'ImageFile']
contour1.Isosurfaces = [5.75]
contour1.PointMergeMethod = 'Uniform Binning'
contour1.UpdatePipeline()

In [ ]:
from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(view)
w = display(pvdisp)

def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(pvdisp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

# save animage on the client-side
#SaveImage("/dev/shm/foo2.png")

In [ ]:
contour1Display = Show(contour1, view)

contour1Display.Representation = 'Surface'
contour1Display.ColorArrayName = ['POINTS', '']

In [ ]:
Hide(contour1)

In [ ]:
#LoadPlugin("/apps/daint/UES/jenkins/7.0.UP02-20.11/gpu/easybuild/software/ParaView/5.9.0-CrayGNU-20.11-EGL-python3/lib64/paraview-5.9/plugins/AcceleratedAlgorithms/AcceleratedAlgorithms.so", ns=globals())
fname = "/apps/pilatus/UES/jenkins/1.4.0/software/ParaView/5.10.0-cpeCray-21.12-OSMesa/lib64/paraview-5.10/plugins/AcceleratedAlgorithms/AcceleratedAlgorithms.so"
fname = "/apps/daint/UES/jenkins/7.0.UP03/21.09/daint-gpu/software/ParaView/5.10.0-CrayGNU-21.09-EGL/lib/paraview-5.10/plugins/AcceleratedAlgorithms/AcceleratedAlgorithms.so"
LoadPlugin(fname, ns=globals())

Will now use a fully multi-threaded filter

https://vtk.org/doc/nightly/html/classvtkFlyingEdges3D.html#details

This class has been threaded with vtkSMPTools. Using TBB or other non-sequential type may improve performance significantly.

In [ ]:
%%time
flyingEdges3D1 = FlyingEdges3D(registrationName='FlyingEdges3D1', Input=reader)
flyingEdges3D1.ContourBy = ['POINTS', 'ImageFile']
flyingEdges3D1.Isosurfaces = [5.75]

In [ ]:
contour2Display = Show(flyingEdges3D1, view)

contour2Display.Representation = 'Surface'
contour2Display.ColorArrayName = ['POINTS', '']

In [ ]:
SaveImage("/dev/shm/foo2.png")